In [1]:
from unstructured.partition.pdf import partition_pdf
from pydantic import BaseModel
from typing import Any
import pickle

import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
pkls = ["./AMD.10K.2023.pdf-0.pkl", "./BABA.10K.2023.pdf-1.pkl", "./IBM.10K.2023.pdf-2.pkl", "./UBER.10K.2023.pdf-3.pkl", "AAPL.10K.2023.pdf-4.pkl"]
raw_pdf_elements = []
for pkl in pkls:
  with open(f"{pkl}", 'rb') as f:
    raw_pdf_elements.append(pickle.load(f))

In [3]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = [[], [], [], [], []]
for raw_pdf_element,categorized_element in zip(raw_pdf_elements,categorized_elements):
    for element in raw_pdf_element:
        if "unstructured.documents.elements.Table" in str(type(element)):
            categorized_element.append(Element(type="table", text=str(element.metadata.text_as_html)))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            categorized_element.append(Element(type="text", text=str(element)))


In [4]:
# Tables
# table_elements = [[], [], [], [], []]
# for table_element,categorized_element in zip(table_elements,categorized_elements):
#   for e in categorized_element:
#     if e.type=="table":
#       table_element.append(e)
  # print(len(table_element))
table_elements = [ [e for e in categorized_element if e.type == "table"] for categorized_element in categorized_elements ]

# Text
# text_elements = [[], [], [], [], []]
# for text_element,categorized_element in zip(text_elements,categorized_elements):
#   for e in categorized_element:
#     if e.type=="text":
#       text_element.append(e)
#   print(len(text_element))
text_elements = [ [e for e in categorized_element if e.type == "text"] for categorized_element in categorized_elements ]


In [14]:
table_elements[4][4].text

'<table><thead><th rowspan="3">Periods July 2, 2023 to August 5, 2023: Open market and privately negotiated purchases</th><th colspan="2">Total Number of Shares Purchased</th><th colspan="2">Average Price Paid Per Share</th><th colspan="2" rowspan="3">of Shares Purchased as Part of Publicly Announced Plans or Programs 33,864 Yet</th><th rowspan="3">Approximate Dollar Value of Shares That May Be Purchased Under the Plans or Programs ()</th></thead><thead><th></th><th></th><th></th><th></th><th></th><th colspan="2"></th><th></th></thead><thead><th></th><th>33,864</th><th></th><th>$</th><th>191.62</th><th colspan="2"></th><th></th></thead><tr><td colspan="8">August 6, 2023 to September 2, 2023:</td></tr><tr><td>August 2023 ASRs</td><td>22,085</td><td>@</td><td></td><td>@</td><td>22,085</td><td>@</td><td></td></tr><tr><td>Open market and privately negotiated purchases</td><td>30,299</td><td></td><td>$</td><td>178.99</td><td>30,299</td><td></td><td></td></tr><tr><td colspan="8">September 3,

In [2]:
import pickle
with open("./table_summaries.pkl", 'rb') as f:
  testpkl = pickle.load(f)

In [4]:
testpkl[0]

["The table lists various sections of a company's annual report, including business, risk factors, unresolved staff comments, properties, legal proceedings, mine safety disclosures, market for common equity, management's discussion and analysis, market risk disclosure, financial statements, changes in accountants, controls and procedures, disclosures regarding foreign jurisdictions, directors and governance, executive compensation, security ownership, related transactions, accounting fees, and exhibits.",
 'The table shows the repurchases of shares during each fiscal quarter of 2022, along with the total number of shares repurchased and the average price paid per share. It also includes the total number of shares repurchased as part of the publicly announced program and the maximum dollar value of shares that may yet be purchased under the program. The last fiscal quarter of 2022 is also included in the table.',
 'The table shows the performance of Advanced Micro Devices, Inc., the S&P